In [1]:
import tensorflow as tf

In [2]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x27f876fae88>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x27f876fbe88>)

In [3]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

In [4]:
def try_gpu(i=0):
    if len(tf.config.experimental.list_physical_devices('GPU')) >= i+1:
        return tf.device(f'/GPU:{i}')
    return tf.device('/CPU:0')

def try_all_gpus():
    num_of_all_gpus = len(tf.config.experimental.list_physical_devices('GPU'))
    devices = [tf.device(f'/GPU:{i}') for i in range(num_of_all_gpus)]
    return devices or [tf.device('/CPU:0')]

try_gpu(), try_gpu(10), try_all_gpus()

(<tensorflow.python.eager.context._EagerDeviceContext at 0x280562f8b08>,
 [<tensorflow.python.eager.context._EagerDeviceContext at 0x280562f8e88>])

In [5]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:CPU:0'

In [6]:
with try_gpu():
    X = tf.ones((2, 3))
X

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)>

In [7]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    net = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1)
    ])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [8]:
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.7552881],
       [0.7552881]], dtype=float32)>

In [9]:
net.layers[0].weights[0].device, net.layers[0].weights[1].device

('/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:0')